In [ ]:
# Load libraries
from aiida_openbis.utils import bisutils

In [ ]:
# Connect to openBIS
session = bisutils.log_in(bisurl="openbis", bisuser="admin", bispasswd="changeit")

In [12]:
# Classes
class OpenBISDatabase:
    def __init__(self):
        self.session = None
        self.schema = {
            "spaces": [], 
            "property_types": [], 
            "object_types": [],
            "experiment_types": [],
            "vocabularies": [],
            "dataSetTypes": [],
        }

    def connect_to_openbis(self, bisurl: str, bisuser: str, bispasswd: str):
        self.session = bisutils.log_in(
            bisurl = bisurl, 
            bisuser = bisuser, 
            bispasswd = bispasswd
        )
    
    def extract_spaces(self):
        spaces = self.session.get_spaces()

        for space in spaces:
            space_metadata = {"code": space.code}
            self.schema["spaces"].append(space_metadata)

    def extract_projects(self):
        for index, space in enumerate(self.schema["spaces"]):
            projects = self.session.get_projects(space = space["code"])
            self.schema["spaces"][index]["projects"] = []

            for project in projects:
                project_metadata = {
                    "code": project.code
                }
                self.schema["spaces"][index]["projects"].append(project_metadata)
    
    def extract_experiments(self):
        for index_space, space in enumerate(self.schema["spaces"]):
            for index_project, project in enumerate(space["projects"]):
                experiments = self.session.get_experiments(
                    space = space["code"], 
                    project = project["code"]
                )
                self.schema["spaces"][index_space]["projects"][index_project]["experiments"] = []

                for experiment in experiments:
                    experiment_metadata = {
                        "code": experiment.code, 
                        "type": experiment.type.code,
                        "properties": experiment.props.all()
                    }
                    self.schema["spaces"][index_space]["projects"][index_project]["experiments"].append(experiment_metadata)
                    
    def extract_objects(self):
        for index_space, space in enumerate(self.schema["spaces"]):
            for index_project, project in enumerate(space["projects"]):
                for index_experiment, experiment in enumerate(project["experiments"]):
                    objects = self.session.get_samples(
                        space = space["code"], 
                        experiment = f"/{space['code']}/{project['code']}/{experiment['code']}"
                    )
                    self.schema["spaces"][index_space]["projects"][index_project]["experiments"][index_experiment]["objects"] = []

                    for object in objects:
                        object = self.session.get_object(object.permId) #TODO: Correct this easy fix once the bug of getting parents from items from get_objects() is fixed

                        object_parents_identifiers = []
                        object_children_identifiers = []
                        object_datasets_metadata = []

                        for parent in object.get_parents():
                            object_parents_identifiers.append(parent.identifier)
                        
                        for child in object.get_children():
                            object_children_identifiers.append(child.identifier)

                        object_datasets = object.get_datasets()
                        for dataset in object_datasets:
                            dataset.data["dataStore"]["downloadUrl"] = 'https://openbis/'
                            dataset.download(destination = 'openBIS_datasets')
                            object_datasets_metadata.append({"type": dataset.type.code, "folderpath": f"openBIS_datasets/{dataset.code}"})

                        object_metadata = {
                            "code": object.code, 
                            "type": object.type.code,
                            "properties": object.props.all(),
                            "parents": object_parents_identifiers,
                            "children": object_children_identifiers,
                            "datasets": object_datasets_metadata
                        }
                        self.schema["spaces"][index_space]["projects"][index_project]["experiments"][index_experiment]["objects"].append(object_metadata)


In [11]:
from aiida_openbis.utils import bisutils
session = bisutils.log_in(bisurl="openbis", bisuser="admin", bispasswd="changeit")
dataset = session.get_samples(code = 'SPMI47')[0].get_datasets()
# dataset.data["dataStore"]["downloadUrl"] = 'https://openbis/'
# dataset.download(destination = 'openBIS_File')
dataset[0].props

property,value,description,type,mandatory
$name,,Name,VARCHAR,False
notes,,Notes,MULTILINE_VARCHAR,False
$xmlcomments,,Comments log,XML,False


In [13]:
openbis_database = OpenBISDatabase()
openbis_database.connect_to_openbis("openbis", "admin", "changeit")
openbis_database.extract_spaces()
openbis_database.extract_projects()
openbis_database.extract_experiments()
openbis_database.extract_objects()
x = openbis_database.schema
x["spaces"][10]

{'code': 'MATERIALS',
 'projects': [{'code': 'MOLECULES',
   'experiments': [{'code': 'MOLECULES_EXP_1',
     'type': 'COLLECTION',
     'properties': {'$name': 'Precursors',
      '$default_object_type': 'MOLECULE',
      '$default_collection_view': None},
     'objects': [{'code': 'MOL19',
       'type': 'MOLECULE',
       'properties': {'$name': '524',
        'empa-number': None,
        'batch': 'A',
        'iupac-name': None,
        'sum-formula': None,
        'smiles': None,
        'cas-number': None,
        'hazardous': None,
        'hazardous-specification': None,
        'evaporation-temperature': None,
        'fridge': None,
        'no-light': None,
        'dry': None,
        'no-ox': None,
        'other-storage-condition': None,
        'other-storage-condition-specification': None,
        'molecule-vials': None,
        'molecule-crucibles': None,
        'amount': None,
        'receive-date': None,
        'comments': None},
       'parents': [],
       'chil

In [14]:
x["spaces"][10]

{'code': 'INSTITUTIONS',
 'projects': [{'code': 'INSTITUTIONS',
   'experiments': [{'code': 'INSTITUTIONS_EXP_1',
     'type': 'COLLECTION',
     'properties': {'$name': 'Institutions',
      '$default_object_type': 'INSTITUTION',
      '$default_collection_view': None},
     'objects': [{'code': 'INST5',
       'type': 'INSTITUTION',
       'properties': {'$name': 'EMPA', 'address': None},
       'parents': [],
       'children': ['/PEOPLE/PEOPLE/AUTH65',
        '/PEOPLE/PEOPLE/AUTH66',
        '/INSTITUTIONS/ROOMS/ROOM54',
        '/INSTITUTIONS/ROOMS/ROOM55'],
       'datasets': [{'type': 'ELN_PREVIEW',
         'folderpath': 'openBIS_datasets/20231106215919638-116'}]},
      {'code': 'INST6',
       'type': 'INSTITUTION',
       'properties': {'$name': 'Max Planck Institute for Polymer Research',
        'address': None},
       'parents': [],
       'children': ['/PEOPLE/PEOPLE/AUTH67'],
       'datasets': [{'type': 'ELN_PREVIEW',
         'folderpath': 'openBIS_datasets/20231106

In [ ]:
session.get_experiments()[0].type.code

In [ ]:
for _,sample in enumerate(session.get_samples()):
    sample.get_parents()

In [ ]:
spaces_df = session.get_spaces().df

projects = []
for space_code in spaces_df.code:
    projects.append(session.get_projects(space = space_code).df)

projects[5]

In [ ]:
session.get_spaces()
session.get_projects()
# session.get_experiments()
# session.get_samples()
# session.get_datasets()
# session.get_sample_type("STM_SIMULATION").get_property_assignments()
# session.get_property_types().df